In [4]:
import openai
import time

In [ ]:
# demo
import base64
import requests


# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


# your key
api_key = ''

image_path = "./dataset/wildfire/valid/wildfire/-64.7254,48.84592.jpg"
base64_image = encode_image(image_path)
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

text = "Had wildfire happened in this picture? Please give your answer and explain it."

payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

output = response.json()
print(output['choices'][0]['message']['content'])

Direct question

In [7]:
text = "Had wildfire happened in this picture? If yes, give me 1, else 0. Just tell me 0 or 1 without any other text."

import base64
import requests
import os


# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = ''

def get_all_filenames(directory_path):
    # Get a list of all files in the directory
    filenames = os.listdir(directory_path)
    
    # Filter out directories (if you only want files)
    filenames = [f for f in filenames if os.path.isfile(os.path.join(directory_path, f))]
    
    return filenames

# Example usage:
directory_path = './dataset/wildfire/test/wildfire/'
filenames_fire = get_all_filenames(directory_path)
directory_path_nofire = './dataset/wildfire/test/nowildfire/'
filenames_nofire = get_all_filenames(directory_path_nofire)

num = 20
l = []
for i in range(num):
  l.append(directory_path+filenames_fire[i])
for i in range(num):
  l.append(directory_path_nofire+filenames_nofire[i])

pred = []
truth = [1]*num + [0]*num


for fn in l:
  base64_image = encode_image(fn)
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o-mini",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": text
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  output = response.json()
  pred.append(int(output['choices'][0]['message']['content']))

FS

In [ ]:
# background, role, few-shot

text1 = """Background: This dataset contains satellite images about wildfire in Canada. This dataset uses Longitude and Latitude coordinates for each wildfire spot (> 0.01 acres burned) found on the dataset.
        Goal: We extracted satellite images of those areas predict whether an area is at risk of a wildfire or not.
        Tips: Areas with wildfire may demonstrate different color in satellite images.
        Question: You are a careful professor of forestry, and good at observing satellite images.
        Now, let's think step by step, and tell me, Had wildfire happened in this picture?
        If yes, give me 1, else 0. Just tell me 0 or 1 without any other text.
        
        Following the target image, I will give you several examples of satellite images with 'yes' or 'no' to specify if wildfire exists.
        This truths of the given 10 examplary images are ['yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no'].
        For images without wildfire, crowded buildings may appear in them.
        
        Now, the first image is the target one, and the remaining ones are examples:
        
"""

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = ''

def get_all_filenames(directory_path):
    # Get a list of all files in the directory
    filenames = os.listdir(directory_path)
    
    # Filter out directories (if you only want files)
    filenames = [f for f in filenames if os.path.isfile(os.path.join(directory_path, f))]
    
    return filenames

# Example usage:
directory_path = './dataset/wildfire/test/wildfire/'
filenames_fire = get_all_filenames(directory_path)
directory_path_nofire = './dataset/wildfire/test/nowildfire/'
filenames_nofire = get_all_filenames(directory_path_nofire)

num = 20
fs_n = 5
l = []
fs_l = []
for i in range(num):
  l.append(directory_path+filenames_fire[i])
for i in range(num):
  l.append(directory_path_nofire+filenames_nofire[i])
  
for i in range(fs_n):
    fs_l.append(encode_image(directory_path+filenames_fire[-i-1]))
for i in range(fs_n):
    fs_l.append(encode_image(directory_path_nofire+filenames_nofire[-i-1]))

pred = []
truth = [1]*num + [0]*num


for fn in l:
  base64_image = encode_image(fn)
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o-mini",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": text
          }] + [
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpg;base64,{base64_image}"
            }
          }] + [
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpg;base64,{fs_l[i]}"
            }
          } for i in range(len(fs_l))
        ]
      }
    ],
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  output = response.json()
  pred.append(int(output['choices'][0]['message']['content']))